In [6]:
SELECT * 
FROM 'summer.csv' AS Summer_Medals;

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
0,1896,Athens,Aquatics,Swimming,HAJOS Alfred,HUN,Men,100M Freestyle,Gold
1,1896,Athens,Aquatics,Swimming,HERSCHMANN Otto,AUT,Men,100M Freestyle,Silver
2,1896,Athens,Aquatics,Swimming,DRIVAS Dimitrios,GRE,Men,100M Freestyle For Sailors,Bronze
3,1896,Athens,Aquatics,Swimming,MALOKINIS Ioannis,GRE,Men,100M Freestyle For Sailors,Gold
4,1896,Athens,Aquatics,Swimming,CHASAPIS Spiridon,GRE,Men,100M Freestyle For Sailors,Silver
...,...,...,...,...,...,...,...,...,...
31160,2012,London,Wrestling,Wrestling Freestyle,JANIKOWSKI Damian,POL,Men,Wg 84 KG,Bronze
31161,2012,London,Wrestling,Wrestling Freestyle,REZAEI Ghasem Gholamreza,IRI,Men,Wg 96 KG,Gold
31162,2012,London,Wrestling,Wrestling Freestyle,TOTROV Rustam,RUS,Men,Wg 96 KG,Silver
31163,2012,London,Wrestling,Wrestling Freestyle,ALEKSANYAN Artur,ARM,Men,Wg 96 KG,Bronze


In [7]:
SELECT
  *,
  -- Assign numbers to each row
  ROW_NUMBER() OVER() AS Row_N
FROM Summer_Medals
ORDER BY Row_N ASC;

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal,Row_N
0,1896,Athens,Aquatics,Swimming,HAJOS Alfred,HUN,Men,100M Freestyle,Gold,1
1,1896,Athens,Aquatics,Swimming,HERSCHMANN Otto,AUT,Men,100M Freestyle,Silver,2
2,1896,Athens,Aquatics,Swimming,DRIVAS Dimitrios,GRE,Men,100M Freestyle For Sailors,Bronze,3
3,1896,Athens,Aquatics,Swimming,MALOKINIS Ioannis,GRE,Men,100M Freestyle For Sailors,Gold,4
4,1896,Athens,Aquatics,Swimming,CHASAPIS Spiridon,GRE,Men,100M Freestyle For Sailors,Silver,5
...,...,...,...,...,...,...,...,...,...,...
31160,2012,London,Wrestling,Wrestling Freestyle,JANIKOWSKI Damian,POL,Men,Wg 84 KG,Bronze,31161
31161,2012,London,Wrestling,Wrestling Freestyle,REZAEI Ghasem Gholamreza,IRI,Men,Wg 96 KG,Gold,31162
31162,2012,London,Wrestling,Wrestling Freestyle,TOTROV Rustam,RUS,Men,Wg 96 KG,Silver,31163
31163,2012,London,Wrestling,Wrestling Freestyle,ALEKSANYAN Artur,ARM,Men,Wg 96 KG,Bronze,31164


In [8]:
--- Numbering Olympic games in ascending order

SELECT
  Year,

  -- Assign numbers to each year
  ROW_NUMBER() OVER() AS Row_N
FROM (
  SELECT distinct year
  FROM Summer_Medals
  ORDER BY Year ASC
) AS Years
ORDER BY Year ASC;

,Year,Row_N
0,1896,1
1,1900,2
2,1904,3
3,1908,4
4,1912,5
5,1920,6
6,1924,7
7,1928,8
8,1932,9
9,1936,10


In [9]:
--- Numbering Olympic games in descending order


SELECT
  Year,
  -- Assign the lowest numbers to the most recent years
  Row_number() OVER (order by year DESC) AS Row_N
FROM (
  SELECT DISTINCT Year
  FROM Summer_Medals
) AS Years
ORDER BY Year;

,Year,Row_N
0,1896,27
1,1900,26
2,1904,25
3,1908,24
4,1912,23
5,1920,22
6,1924,21
7,1928,20
8,1932,19
9,1936,18


In [10]:
--- Numbering Olympic athletes by medals earned

SELECT
  -- Count the number of medals each athlete has earned
  athlete,
  count(medal) AS Medals
FROM Summer_Medals
GROUP BY Athlete
ORDER BY Medals DESC;


,Athlete,Medals
0,PHELPS Michael,22
1,LATYNINA Larisa,18
2,ANDRIANOV Nikolay,15
3,ONO Takashi,13
4,MANGIAROTTI Edoardo,13
...,...,...
22757,BURROUGHS Jordan Ernest,1
22758,KIM Hyeonwoo,1
22759,GAJIYEV Danyal,1
22760,ALEKSANYAN Artur,1


In [11]:
--- Reigning weightlifting champions

SELECT
  -- Return each year's champions' countries
  year,
  country AS champion
FROM Summer_Medals
WHERE
  Discipline = 'Weightlifting' AND
  Event = '69KG' AND
  Gender = 'Men' AND
  Medal = 'Gold';

,Year,champion
0,2000,BUL
1,2004,CHN
2,2008,CHN
3,2012,CHN


In [12]:
--- Reigning champions by gender

WITH Tennis_Gold AS (
  SELECT DISTINCT
    Gender, Year, Country
  FROM Summer_Medals
  WHERE
    Year >= 2000 AND
    Event = 'Javelin Throw' AND
    Medal = 'Gold')

SELECT
  Gender, Year,
  Country AS Champion,
  -- Fetch the previous year's champion by gender
lag(country) OVER (PARTITION BY gender
            ORDER BY year ASC) AS Last_Champion
FROM Tennis_Gold
ORDER BY Gender ASC, Year ASC;


,Gender,Year,Champion,Last_Champion
0,Men,2000,CZE,None
1,Men,2004,NOR,CZE
2,Men,2008,NOR,NOR
3,Men,2012,TTO,NOR
4,Women,2000,NOR,None
5,Women,2004,CUB,NOR
6,Women,2008,CZE,CUB
7,Women,2012,CZE,CZE


In [ ]:
--- Reigning champions by gender and event

WITH Athletics_Gold AS (
  SELECT DISTINCT
    Gender, Year, Event, Country
  FROM Summer_Medals
  WHERE
    Year >= 2000 AND
    Discipline = 'Athletics' AND
    Event IN ('100M', '10000M') AND
    Medal = 'Gold')

SELECT
  Gender, Year, Event,
  Country AS Champion,
  -- Fetch the previous year's champion by gender and event
  lag(country) OVER (partition by gender, event
            ORDER BY Year ASC) AS Last_Champion
FROM Athletics_Gold
ORDER BY Event ASC, Gender ASC, Year ASC;

In [13]:
--- Future gold medalists

WITH Discus_Medalists AS (
  SELECT DISTINCT
    Year,
    Athlete
  FROM Summer_Medals
  WHERE Medal = 'Gold'
    AND Event = 'Discus Throw'
    AND Gender = 'Women'
    AND Year >= 2000)

SELECT
  -- For each year, fetch the current and future medalists
  Year,
  Athlete,
  LEAD(Athlete, 3) OVER (ORDER BY Year ASC) AS Future_Champion
FROM Discus_Medalists
ORDER BY Year ASC;

,Year,Athlete,Future_Champion
0,2000,ZVEREVA Ellina,PERKOVIC Sandra
1,2004,SADOVA Natalya,None
2,2008,BROWN TRAFTON Stephanie,None
3,2012,PERKOVIC Sandra,None


In [14]:
--- First athlete by name

WITH All_Male_Medalists AS (
  SELECT DISTINCT
    Athlete
  FROM Summer_Medals
  WHERE Medal = 'Gold'
    AND Gender = 'Men')

SELECT
  -- Fetch all athletes and the first athlete alphabetically
  athlete,
  first_value(athlete) OVER (
    ORDER BY athlete  ASC
  ) AS First_Athlete
FROM All_Male_Medalists;

,Athlete,First_Athlete
0,AABYE Edgar,AABYE Edgar
1,AALTONEN Paavo Johannes,AABYE Edgar
2,AAS Thomas Valentin,AABYE Edgar
3,ABALMASAU Aliaksei,AABYE Edgar
4,ABALO Luc,AABYE Edgar
...,...,...
6240,ÖRVIG Thor,AABYE Edgar
6241,ÖSTERVOLD Henrik,AABYE Edgar
6242,ÖSTERVOLD Jan Olsen,AABYE Edgar
6243,ÖSTERVOLD Kristian Olsen,AABYE Edgar


In [15]:
--- Last country by name

WITH Hosts AS (
  SELECT DISTINCT Year, City
    FROM Summer_Medals)

SELECT
  Year,
  City,
  -- Get the last city in which the Olympic games were held
  last_value(city) OVER (
   ORDER BY year ASC
   RANGE BETWEEN
     UNBOUNDED PRECEDING AND
     UNBOUNDED FOLLOWING
  ) AS Last_City
FROM Hosts
ORDER BY Year ASC;


,Year,City,Last_City
0,1896,Athens,London
1,1900,Paris,London
2,1904,St Louis,London
3,1908,London,London
4,1912,Stockholm,London
5,1920,Antwerp,London
6,1924,Paris,London
7,1928,Amsterdam,London
8,1932,Los Angeles,London
9,1936,Berlin,London


In [16]:
--- Ranking athletes by medals earned

WITH Athlete_Medals AS (
  SELECT
    Athlete,
    COUNT(*) AS Medals
  FROM Summer_Medals
  GROUP BY Athlete)

SELECT
  Athlete,
  Medals,
  -- Rank athletes by the medals they've won
  Rank() OVER (ORDER BY medals DESC) AS Rank_N
FROM Athlete_Medals
ORDER BY Medals DESC;

,Athlete,Medals,Rank_N
0,PHELPS Michael,22,1
1,LATYNINA Larisa,18,2
2,ANDRIANOV Nikolay,15,3
3,MANGIAROTTI Edoardo,13,4
4,ONO Takashi,13,4
...,...,...,...
22757,GHASEMI Komeil,1,5267
22758,VOLOSOVA Lubov,1,5267
22759,MATSUMOTO Ryutaro,1,5267
22760,LORINCZ Tamas,1,5267


In [17]:
--- Ranking athletes from multiple countries

WITH Athlete_Medals AS (
  SELECT
    Country, Athlete, COUNT(*) AS Medals
  FROM Summer_Medals
  WHERE
    Country IN ('JPN', 'KOR')
    AND Year >= 2000
  GROUP BY Country, Athlete
  HAVING COUNT(*) > 1)

SELECT
  Country,
  -- Rank athletes in each country by the medals they've won
  Athlete,
  DENse_Rank() OVER (PARTITION BY country
                ORDER BY Medals DESC) AS Rank_N
FROM Athlete_Medals
ORDER BY Country ASC, RANK_N ASC;

,Country,Athlete,Rank_N
0,JPN,KITAJIMA Kosuke,1
1,JPN,UCHIMURA Kohei,2
2,JPN,TACHIBANA Miya,3
3,JPN,TAKEDA Miho,3
4,JPN,TOMITA Hiroyuki,4
...,...,...,...
69,KOR,KIM Minjae,4
70,KOR,KI Bo Bae,4
71,KOR,HA Tae-Kwon,4
72,KOR,KIM Kyungah,4


In [18]:
--- Paging events

WITH Events AS (
  SELECT DISTINCT Event
  FROM Summer_Medals)
  
SELECT
  --- Split up the distinct events into 111 unique groups
  Event,
  ntile(111) OVER (ORDER BY Event ASC) AS Page
FROM Events
ORDER BY Event ASC;

,Event,Page
0,+ 100KG,1
1,+ 100KG (Heavyweight),1
2,+ 100KG (Super Heavyweight),1
3,+ 105KG,1
4,+ 108KG Total (Super Heavyweight),1
...,...,...
661,York Round (100Y - 80Y - 60Y),111
662,Épée Amateurs And Masters,111
663,Épée Individual,111
664,Épée Masters,111


In [19]:
--- Top, middle, and bottom thirds

WITH Athlete_Medals AS (
  SELECT Athlete, COUNT(*) AS Medals
  FROM Summer_Medals
  GROUP BY Athlete
  HAVING COUNT(*) > 1)
  
SELECT
  Athlete,
  Medals,
  -- Split athletes into thirds by their earned medals
  ntile(3) over (order by medals DESC) AS Third
FROM Athlete_Medals
ORDER BY Medals DESC, Athlete ASC;

,Athlete,Medals,Third
0,PHELPS Michael,22,1
1,LATYNINA Larisa,18,1
2,ANDRIANOV Nikolay,15,1
3,MANGIAROTTI Edoardo,13,1
4,ONO Takashi,13,1
...,...,...,...
5261,ZVEREVA Ellina,2,3
5262,ZWERVER Ronald,2,3
5263,ZWOLLE Hendrik Jan,2,3
5264,ZYKINA Olesya,2,2


In [20]:
--- Running totals of athlete medals

WITH Athlete_Medals AS (
  SELECT
    Athlete, COUNT(*) AS Medals
  FROM Summer_Medals
  WHERE
    Country = 'USA' AND Medal = 'Gold'
    AND Year >= 2000
  GROUP BY Athlete)

SELECT
  -- Calculate the running total of athlete medals
  Athlete,
  Medals,
  sum(Medals) OVER (ORDER BY athlete ASC) AS Max_Medals
FROM Athlete_Medals
ORDER BY Athlete ASC;

,Athlete,Medals,Max_Medals
0,ABDUR-RAHIM Shareef,1,1.0
1,ABERNATHY Brent,1,2.0
2,ADRIAN Nathan,3,5.0
3,AHRENS Chris,1,6.0
4,AINSWORTH Kurt,1,7.0
...,...,...,...
362,WOLTERS Kara,1,513.0
363,WYLDE Peter,1,514.0
364,YOUNG Ernie,1,515.0
365,YOUNG Tim,1,516.0


In [21]:
--- Maximum country medals by year

WITH Country_Medals AS (
  SELECT
    Year, Country, COUNT(*) AS Medals
  FROM Summer_Medals
  WHERE
    Country IN ('CHN', 'KOR', 'JPN')
    AND Medal = 'Gold' AND Year >= 2000
  GROUP BY Year, Country)

SELECT
  -- Return the max medals earned so far per country
  Medals,
  Country,
  Year,
  max(Medals) OVER (PARTITION BY Country
                ORDER BY Year ASC) AS Max_Medals
FROM Country_Medals
ORDER BY Country ASC, Year ASC;

,Medals,Country,Year,Max_Medals
0,39,CHN,2000,39
1,52,CHN,2004,52
2,74,CHN,2008,74
3,56,CHN,2012,74
4,5,JPN,2000,5
5,21,JPN,2004,21
6,23,JPN,2008,23
7,7,JPN,2012,23
8,12,KOR,2000,12
9,14,KOR,2004,14


In [22]:
--- Minimum country medals by year

WITH France_Medals AS (
  SELECT
    Year, COUNT(*) AS Medals
  FROM Summer_Medals
  WHERE
    Country = 'FRA'
    AND Medal = 'Gold' AND Year >= 2000
  GROUP BY Year)

SELECT
  Year,
  Medals,
  Min(Medals) OVER (ORDER BY year ASC) AS Min_Medals
FROM France_Medals
ORDER BY Year ASC;

,Year,Medals,Min_Medals
0,2000,22,22
1,2004,21,21
2,2008,25,21
3,2012,30,21


In [23]:
--- Moving maximum of Scandinavian athletes' medals

WITH Scandinavian_Medals AS (
  SELECT
    Year, COUNT(*) AS Medals
  FROM Summer_Medals
  WHERE
    Country IN ('DEN', 'NOR', 'FIN', 'SWE', 'ISL')
    AND Medal = 'Gold'
  GROUP BY Year)

SELECT
  -- Select each year's medals
  Year,
  Medals,
  -- Get the max of the current and next years'  medals
  max(Medals) OVER (ORDER BY Year ASC
             ROWS BETWEEN current row
             AND 1 following) AS Max_Medals
FROM Scandinavian_Medals
ORDER BY Year ASC;

,Year,Medals,Max_Medals
0,1896,1,1
1,1900,1,77
2,1908,77,141
3,1912,141,159
4,1920,159,159
5,1924,48,48
6,1928,24,24
7,1932,17,17
8,1936,15,54
9,1948,54,54


In [24]:
--- Moving maximum of Chinese athletes' medals

WITH Chinese_Medals AS (
  SELECT
    Athlete, COUNT(*) AS Medals
  FROM Summer_Medals
  WHERE
    Country = 'CHN' AND Medal = 'Gold'
    AND Year >= 2000
  GROUP BY Athlete)

SELECT
  -- Select the athletes and the medals they've earned
  Athlete,
  Medals,
  -- Get the max of the last two and current rows' medals 
  max(Medals) OVER (ORDER BY Athlete ASC
            ROWS BETWEEN 2 preceding
            AND current row) AS Max_Medals
FROM Chinese_Medals
ORDER BY Athlete ASC;

,Athlete,Medals,Max_Medals
0,CAI Yalin,1,1
1,CAI Yun,1,1
2,CAO Lei,1,1
3,CAO Yuan,1,1
4,CHEN Ding,1,1
...,...,...,...
155,ZHOU Lulu,1,1
156,ZHOU Suhong,1,1
157,ZHU Qinan,1,1
158,ZOU Kai,5,5


In [25]:
--- Moving average of Russian medals

WITH Russian_Medals AS (
  SELECT
    Year, COUNT(*) AS Medals
  FROM Summer_Medals
  WHERE
    Country = 'RUS'
    AND Medal = 'Gold'
    AND Year >= 1980
  GROUP BY Year)

SELECT
  Year, Medals,
  --- Calculate the 3-year moving average of medals earned
  avg(Medals) OVER
    (ORDER BY Year ASC
     ROWS BETWEEN
     2 preceding AND current row) AS Medals_MA
FROM Russian_Medals
ORDER BY Year ASC;

,Year,Medals,Medals_MA
0,1996,36,36.000000
1,2000,66,51.000000
2,2004,47,49.666667
3,2008,43,52.000000
4,2012,47,45.666667


In [26]:
--- Moving total of countries' medals

WITH Country_Medals AS (
  SELECT
    Year, Country, COUNT(*) AS Medals
  FROM Summer_Medals
  GROUP BY Year, Country)

SELECT
  Year, Country, Medals,
  -- Calculate each country's 3-game moving total
  sum(Medals) OVER
    (PARTITION BY Country
     ORDER BY Year ASC
     ROWS BETWEEN
     2 preceding AND current row) AS Medals_MA
FROM Country_Medals
ORDER BY Country ASC, Year ASC;

,Year,Country,Medals,Medals_MA
0,2008,AFG,1,1.0
1,2012,AFG,1,2.0
2,1988,AHO,1,1.0
3,1984,ALG,2,2.0
4,1992,ALG,2,4.0
...,...,...,...,...
1153,2008,ZIM,4,23.0
1154,1896,ZZX,6,6.0
1155,1900,ZZX,34,40.0
1156,1904,ZZX,8,48.0


In [27]:
--- Pivoting with ranking

WITH Country_Awards AS (
  SELECT
    Country,
    Year,
    COUNT(*) AS Awards
  FROM Summer_Medals
  WHERE
    Country IN ('FRA', 'GBR', 'GER')
    AND Year IN (2004, 2008, 2012)
    AND Medal = 'Gold'
  GROUP BY Country, Year)

SELECT
  -- Select Country and Year
  Country,
  Year,
  -- Rank by gold medals earned per year
  Awards :: INTEGER AS rank
FROM Country_Awards
ORDER BY Country ASC, Year ASC;

,Country,Year,rank
0,FRA,2004,21
1,FRA,2008,25
2,FRA,2012,30
3,GBR,2004,17
4,GBR,2008,31
5,GBR,2012,48
6,GER,2004,41
7,GER,2008,42
8,GER,2012,45


In [28]:
--- Country-level subtotals

-- Count the gold medals per country and gender
SELECT
  Country,
  Gender,
  COUNT(*) AS Gold_Awards
FROM Summer_Medals
WHERE
  Year = 2004
  AND Medal = 'Gold'
  AND Country IN ('DEN', 'NOR', 'SWE')
-- Generate Country-level subtotals
GROUP BY Country, ROLLUP(Gender)
ORDER BY Country ASC, Gender ASC;

,Country,Gender,Gold_Awards
0,DEN,Men,4
1,DEN,Women,15
2,DEN,None,19
3,NOR,Men,3
4,NOR,Women,2
5,NOR,None,5
6,SWE,Men,4
7,SWE,Women,1
8,SWE,None,5


In [29]:
--- All group-level subtotals

-- Count the medals per gender and medal type
SELECT
  Gender,
  Medal,
  count(*) AS Awards
FROM Summer_Medals
WHERE
  Year = 2012
  AND Country = 'RUS'
-- Get all possible group-level subtotals
GROUP BY cube(Gender,Medal)
ORDER BY Gender ASC, Medal ASC;

,Gender,Medal,Awards
0,Men,Bronze,34
1,Men,Gold,23
2,Men,Silver,7
3,Men,None,64
4,Women,Bronze,17
5,Women,Gold,24
6,Women,Silver,25
7,Women,None,66
8,None,Bronze,51
9,None,Gold,47


In [30]:
--- Cleaning up results

SELECT
  -- Replace the nulls in the columns with meaningful text
  COALESCE(Country,'All countries') AS Country,
  COALESCE(Gender,'All genders') AS Gender,
  COUNT(*) AS Awards
FROM Summer_Medals
WHERE
  Year = 2004
  AND Medal = 'Gold'
  AND Country IN ('DEN', 'NOR', 'SWE')
GROUP BY ROLLUP(Country, Gender)
ORDER BY Country ASC, Gender ASC;

,Country,Gender,Awards
0,All countries,All genders,29
1,DEN,All genders,19
2,DEN,Men,4
3,DEN,Women,15
4,NOR,All genders,5
5,NOR,Men,3
6,NOR,Women,2
7,SWE,All genders,5
8,SWE,Men,4
9,SWE,Women,1


In [31]:
--- Summarizing results

WITH Country_Medals AS (
  SELECT
    Country,
    COUNT(*) AS Medals
  FROM Summer_Medals
  WHERE Year = 2000
    AND Medal = 'Gold'
  GROUP BY Country),

  Country_Ranks AS (
  SELECT
    Country,
    RANK() OVER (ORDER BY Medals DESC) AS Rank
  FROM Country_Medals
  ORDER BY Rank ASC)

-- Compress the countries column
SELECT STRING_AGG(Country, ', ')
FROM Country_Ranks
-- Select only the top three ranks
WHERE Rank <= 3 ; 

,"string_agg(Country, ', ')"
0,"USA, RUS, AUS"
